In [ ]:
################################  Storage_Optimization.ipynb  ####################################
# Author: Sukhendu Sain
# Description: Main file of codebase. Houses main code
# Data: 23-Nov-2024
#################################################################################

In [ ]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import os, re
import matplotlib.pyplot as plt
# import importlib
# importlib.reload(utils)

# Data Import and Clean

In [ ]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
# df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
# df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
# if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [ ]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~50-60secs

In [ ]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale['Description'] = df_Wholesale['Description'].astype(str)
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR')].reset_index()
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Wholesale['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Wholesale['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Wholesale) # Print the Dataframe
# ~12-15secs

In [ ]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service['Description'] = df_Service['Description'].astype(str)
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR')].reset_index()
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Service['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Service['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Dollars Sold'] = df_Service['Qty Sold'].apply(lambda x: [i for i in str(x).strip().split("      ")][-1])
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Qty Sold'] = df_Service['Qty Sold'].apply(lambda x: "     ".join([i for i in str(x).strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~5-6secs

In [ ]:
#### Read FILE:: (Counter Pad) into Dataframe

# Data Processing & Calculation

### Make a Big Final Dataframe

In [ ]:
# It will have the Columns - 'Part Number', 'Part Desc.', 'Active', 'Sold (Pcs.)', '0Dimensions', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from 3 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = [] # Initialize the New List, which will hole all rows before turning into DF
gParts_PartNos = set(df_Gparts['Svc Part Number']) # Get a Set of all Part Nos. of GParts


# Wholesale
common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Wholesale", ac, df_Wholesale[df_Wholesale['Part Number'] == pn]['Sold'].values[0], 0, 0, False, dp, wd, ht, "", "", "", "", ""])


# Service
common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Service", ac, 0, df_Service[df_Service['* indicates a superseded part\nPart Number'] == pn]['Qty Sold'].values[0], 0, False, dp, wd, ht, "", "", "", "", ""])


# Create the Main Dataframe
df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Desc.', 'Part Category', 'DataSource', 'Active', 'Wholesale Sold', 'Service Sold', "Total Sold", '0Dimensions', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'Num. Storage Required', "Bin Location"]

# Merge the Parts present in both Service and Wholesale (Duplicates)
gbParts = df_Main.groupby('Part#').count()[df_Main.groupby('Part#').count()['Part Desc.'] == 2].index.to_list()
for pn in gbParts:
    df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Wholesale"), "Service Sold"] = df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Service"), "Service Sold"].values[0]
    df_Main = df_Main[(df_Main['Part#'] != pn) | (df_Main['DataSource'] == "Wholesale")]
# Set 0Dimensions
df_Main.loc[(df_Main["Depth"] == 0) | (df_Main["Height"] == 0) | (df_Main["Width"] == 0), "0Dimensions"] = True
# Drop 0Dimensions Rows if drop0Dims
if drop0Dims: df_Main = df_Main[df_Main["0Dimensions"] == False]
# Set Total_Sold
df_Main["Total Sold"] = df_Main["Wholesale Sold"].astype(int) + df_Main["Service Sold"].astype(int)
# Sort by 'Total Sold'
df_Main = df_Main.sort_values('Total Sold', ascending=False)
# Reset Index
df_Main = df_Main.reset_index(drop=True)

utils.print_df(df_Main)

In [ ]:
def part_categorization(df_toBeCategorized):
    # TODO: Add more categories
    for i in range(df_toBeCategorized.shape[0]):
        desc = "-".join(df_toBeCategorized.loc[i, "Part Desc."].split("-")[1:])
        category = ""
        if "rivet" in desc.lower():
            category = "Rivet"
        elif ("blade" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Blade"        
        elif ("arm" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Arm"
        elif ("v-belt" in desc.lower()):
            category = "V-Belt"
        elif ("belt" in desc.lower()) & ("retractor" not in desc.lower()) & ("hole" not in desc.lower()) & ("cover" not in desc.lower()):
            category = "Belt"
        # elif ("hose" in desc.lower()) & ("vent" not in desc.lower()) & ("connect" not in desc.lower() & ("radiator" not in desc.lower()& ("heater" not in desc.lower()):
        #     category = "Hose"

        df_toBeCategorized.loc[i, 'Part Category'] = category

### Apply Zoning

In [ ]:
# TODO: Clarify for other Zones and merge the 2 below functions

## Main Function for Apply Zoning
def Apply_Zoning2(df_toBeZoned, zones, soldColName='Total Sold', zoneColName='Zone'): 
    df_toBeZoned.loc[:, zoneColName] = df_toBeZoned[soldColName].apply(lambda x: next((zone for zone, ratio in zones.items() if x >= ratio), list(zones.keys())[0]))
    df_toBeZoned.loc[df_toBeZoned[soldColName] < 1, zoneColName] = None

In [ ]:
## Run the Apply_Zoning on df_Main
Apply_Zoning2(df_Main, zones, 'Total Sold', 'Zone')

In [ ]:
## Check each Zone's number of Part Numbers
df_Main['Zone'].value_counts()

### Specialty Storage Assignment

In [ ]:
def getSpecialityStorage(pdesc, depth, width, height):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""

    # Parsing for Battery
    if pdesc.split("-")[-1].trim() == "Battery":
        storageType = "Battery Specialty Storage"
        subStorage = "48-inch Deep- 48-inch Wide- 3-Level Sloped Shelving"
    # Parsing for Tire
    elif pdesc.split("-")[-1].trim() == "Tire":
        storageType = "Tire Specialty Storage"
        subStorage = ""           
    # Parsing for Bumper Cover
    elif ("Bumper" in pdesc) & ("Cover" in pdesc):
        storageType = "Bumper Cover Specialty Storage"
        subStorage = ""
    # For Hanging Storage
    elif ((depth > 24) | (width > 24) | (height > 24)) & ((depth < 4) | (width < 4) | (height < 24)):
        storageType = "Hanging Speciality Storage"
        # TODO: Clarify to get the SKU Count and Fix this
        skuCount = 10
        if skuCount <= 10: 
            subStorage = "6-inch Hooks"
        elif 10 < skuCount <= 20:
            subStorage = "12-inch Hooks"

    return storageType, subStorage

### Main Function for Storage Assignment

In [ ]:
def getStorage(zone, pdesc, depth, width, height):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""

    if (zone == "Red Hot") | (zone == "Red"):
        storageType, subStorage = utils.getRedHotStorage(depth, width, height)
    elif zone == "Orange":
        storageType, subStorage = utils.getOrangeStorage(depth, width, height)
    elif zone == "Yellow":
        storageType, subStorage = utils.getYellowStorage(depth, width, height)
    elif zone == "Green":   
        storageType, subStorage = utils.getGreenStorage(depth, width, height)
    elif zone == "Blue":
        storageType, subStorage = utils.getBlueStorage(depth, width, height)

    return storageType, subStorage # Return the Values

#### Apply the Storage Function

In [ ]:
for i in range(df_Main.shape[0]):
    # Set the Dimensions of the Data into Variables
    depth = df_Main.loc[i, "Depth"]
    height = df_Main.loc[i, "Height"]
    width = df_Main.loc[i, "Width"]

    zone = df_Main.loc[i, "Zone"]
    pdesc = df_Main.loc[i, "Part Desc."]

    # If any dimension is zero, set empty Storage
    if df_Main.loc[i, "0Dimensions"] == True:
        df_Main.loc[i, "StorageType"] = ""
        df_Main.loc[i, "SubStorage"] = ""
        continue

    # Set Storage of the Parts
    df_Main.loc[i, "StorageType"], df_Main.loc[i, "SubStorage"] = getStorage(zone, pdesc, depth, width, height)

# TODO & CLARIFICATIŌN --  Calculate Tire Carousel Model Selection:
#df_Main[df_Main['StorageType'] == 'Tire Specialty Storage']['SubStorage'] = 'Standard Carousel Model' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '<=28-inches'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= TIRE_PERCENT2 else 'Large Carousel Model'


In [ ]:
utils.print_df(df_Main)

In [ ]:
df_Main.to_excel('FinalDataset.xlsx', index=False)

In [ ]:
df_Main["StorageType"].value_counts()

In [ ]:
df_Main[(df_Main["0Dimensions"] == True) & (df_Main["StorageType"] == "")]